<a href="https://colab.research.google.com/github/AnaJuvencio/Aprendizado_Profundo_Reconhecimento_Visual/blob/main/Projeto_Aprendizado_Profundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab_Notebooks'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['Projeto_Aprendizado_Profundo.ipynb']


GPU + dependências

In [4]:
!nvidia-smi -L || echo "Sem GPU detectada"
!pip -q install datasets tensorflow matplotlib scikit-learn pandas
import tensorflow as tf
print("GPUs detectadas:", tf.config.list_physical_devices('GPU'))

/bin/bash: line 1: nvidia-smi: command not found
Sem GPU detectada
GPUs detectadas: []


Seeds, mixed precision (opcional)

In [5]:
import os, random, numpy as np, tensorflow as tf
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

try:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy("mixed_float16")
    MIXED = True
except Exception:
    MIXED = False
print("Mixed precision:", MIXED)

Mixed precision: True


Carregar TrashNet + split estratificado (80/10/10)

In [6]:
from datasets import load_dataset

# (RE)CARREGA e REFAZ OS SPLITS — limpa qualquer transform antigo
raw = load_dataset("garythung/trashnet")
splits = raw["train"].train_test_split(test_size=0.2, stratify_by_column="label", seed=SEED)
tmp    = splits["test"].train_test_split(test_size=0.5, stratify_by_column="label", seed=SEED)
ds = {"train": splits["train"], "val": tmp["train"], "test": tmp["test"]}

for k in ds: print(k, ds[k].num_rows)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

dataset-original.zip:   0%|          | 0.00/3.63G [00:00<?, ?B/s]

dataset-resized.zip:   0%|          | 0.00/42.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5054 [00:00<?, ? examples/s]

train 4043
val 505
test 506


Conferir chaves disponíveis

In [7]:
print(ds)
print(ds['train'][0])

{'train': Dataset({
    features: ['image', 'label'],
    num_rows: 4043
}), 'val': Dataset({
    features: ['image', 'label'],
    num_rows: 505
}), 'test': Dataset({
    features: ['image', 'label'],
    num_rows: 506
})}
{'image': <PIL.Image.Image image mode=RGB size=3024x4032 at 0x7B001E579EB0>, 'label': 2}


tf.data “leve” (preprocess on-the-fly)

In [9]:
# ===== PREPROCESS (sem with_transform) — fixa shape ANTES do batch =====
import tensorflow as tf

# 1) Converte para tf.data com batch=1 (cada item vem com shape próprio, sem conflitar)
train_tf = ds["train"].to_tf_dataset(columns=["image"], label_cols=["label"],
                                     shuffle=True,  batch_size=1)
val_tf   = ds["val"].to_tf_dataset(columns=["image"], label_cols=["label"],
                                   shuffle=False, batch_size=1)
test_tf  = ds["test"].to_tf_dataset(columns=["image"], label_cols=["label"],
                                    shuffle=False, batch_size=1)

# 2) Pré-processamento por batch (batch=1): uint8 -> float32 -> normalize -> resize
def preprocess1(x, y):
    x = tf.cast(x, tf.float32)/255.0
    x = tf.image.resize(x, IMG_SIZE)      # garante shape fixo
    return x, y

train_tf = train_tf.map(preprocess1, num_parallel_calls=tf.data.AUTOTUNE)
val_tf   = val_tf.map(preprocess1,   num_parallel_calls=tf.data.AUTOTUNE)
test_tf  = test_tf.map(preprocess1,  num_parallel_calls=tf.data.AUTOTUNE)

# 3) "Desfaz" o batch=1 e re-agrupa no BATCH_SIZE desejado
train_tf = train_tf.unbatch().batch(BATCH_SIZE)
val_tf   = val_tf.unbatch().batch(BATCH_SIZE)
test_tf  = test_tf.unbatch().batch(BATCH_SIZE)

# 4) Augmentação só no conjunto de treino
augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1),
])
train_tf = train_tf.map(lambda x,y: (augment(x, training=True), y),
                        num_parallel_calls=tf.data.AUTOTUNE)

# 5) Prefetch
train_tf = train_tf.prefetch(tf.data.AUTOTUNE)
val_tf   = val_tf.prefetch(tf.data.AUTOTUNE)
test_tf  = test_tf.prefetch(tf.data.AUTOTUNE)

print("pipelines prontos:", train_tf, val_tf, test_tf)

NameError: in user code:

    File "/tmp/ipython-input-3803580253.py", line 15, in preprocess1  *
        x = tf.image.resize(x, IMG_SIZE)      # garante shape fixo

    NameError: name 'IMG_SIZE' is not defined


Balanceamento: class weights

In [ ]:
from collections import Counter
train_labels = [ex["label"] for ex in ds["train"]]
freq = Counter(train_labels); total = sum(freq.values())
class_weight = {c: total/(NUM_CLASSES*freq[c]) for c in freq}
class_weight


CNN baseline (simples)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def build_cnn_baseline(input_shape=(160,160,3), num_classes=NUM_CLASSES, mixed=MIXED):
    m = keras.Sequential(name="cnn_baseline")
    m.add(layers.Conv2D(32, 3, padding="same", use_bias=False, input_shape=input_shape))
    m.add(layers.BatchNormalization()); m.add(layers.ReLU()); m.add(layers.MaxPooling2D())
    m.add(layers.Conv2D(64, 3, padding="same", use_bias=False))
    m.add(layers.BatchNormalization()); m.add(layers.ReLU()); m.add(layers.MaxPooling2D())
    m.add(layers.Conv2D(128, 3, padding="same", use_bias=False))
    m.add(layers.BatchNormalization()); m.add(layers.ReLU()); m.add(layers.MaxPooling2D())
    m.add(layers.Flatten()); m.add(layers.Dense(256, activation="relu")); m.add(layers.Dropout(0.5))
    m.add(layers.Dense(num_classes, activation="softmax", dtype="float32" if mixed else None))
    return m

model = build_cnn_baseline()
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


Callbacks + treino (uma vez, com class_weight)

In [ ]:
ckpt = keras.callbacks.ModelCheckpoint("cnn_baseline_best.keras",
                                       monitor="val_accuracy", save_best_only=True)
early = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=7, restore_best_weights=True)
reduce = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

history = model.fit(
    train_tf,
    validation_data=val_tf,
    epochs=40,
    callbacks=[ckpt, early, reduce],
    class_weight=class_weight
)

Curvas + salvar PNG/CSV

In [ ]:
import matplotlib.pyplot as plt, pandas as pd

hist_df = pd.DataFrame(history.history)
hist_df.to_csv("cnn_baseline_history.csv", index=False)

plt.figure(); plt.plot(hist_df["accuracy"], label="treino"); plt.plot(hist_df["val_accuracy"], label="val")
plt.title("Acurácia"); plt.xlabel("Épocas"); plt.ylabel("Acc"); plt.legend(); plt.tight_layout()
plt.savefig("acc_cnn_baseline.png", dpi=150); plt.show()

plt.figure(); plt.plot(hist_df["loss"], label="treino"); plt.plot(hist_df["val_loss"], label="val")
plt.title("Perda (Loss)"); plt.xlabel("Épocas"); plt.ylabel("Loss"); plt.legend(); plt.tight_layout()
plt.savefig("loss_cnn_baseline.png", dpi=150); plt.show()


Avaliação + matrizes

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import itertools

test_loss, test_acc = model.evaluate(test_tf, verbose=0)
print(f"Acurácia (teste): {test_acc:.4f}")

y_true, y_pred = [], []
for x,y in test_tf:
    p = model.predict(x, verbose=0)
    y_pred.extend(np.argmax(p, axis=1)); y_true.extend(y.numpy())

print("\nRelatório de classificação:\n",
      classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

# salvar relatório por classe
prec, rec, f1, sup = precision_recall_fscore_support(y_true, y_pred, labels=range(NUM_CLASSES))
rep_df = pd.DataFrame({"class": CLASS_NAMES, "precision": prec, "recall": rec, "f1": f1, "support": sup})
rep_df.to_csv("cnn_baseline_class_report.csv", index=False)

# matrizes
cm = confusion_matrix(y_true, y_pred)
cmn = cm.astype("float")/cm.sum(axis=1, keepdims=True)

def plot_cm(M, title, fname, fmt="d"):
    plt.figure(figsize=(6,5))
    plt.imshow(M, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title(title); plt.colorbar()
    ticks = np.arange(NUM_CLASSES)
    plt.xticks(ticks, CLASS_NAMES, rotation=45, ha="right"); plt.yticks(ticks, CLASS_NAMES)
    th = M.max()/2.
    for i, j in itertools.product(range(M.shape[0]), range(M.shape[1])):
        txt = f"{M[i,j]:.2f}" if fmt=="0.2f" else f"{int(M[i,j])}"
        plt.text(j, i, txt, ha="center", color="white" if M[i,j] > th else "black")
    plt.ylabel("Verdadeiro"); plt.xlabel("Predito"); plt.tight_layout()
    plt.savefig(fname, dpi=150); plt.show()

plot_cm(cm,  "Matriz de Confusão (Absoluta)",   "cm_abs_cnn_baseline.png", fmt="d")
plot_cm(cmn, "Matriz de Confusão (Normalizada)","cm_norm_cnn_baseline.png", fmt="0.2f")


Curvas de aprendizado + salvar PNG

In [ ]:
import matplotlib.pyplot as plt, pandas as pd

hist_df = pd.DataFrame(history.history)
hist_df.to_csv("cnn_baseline_history.csv", index=False)

plt.figure(); plt.plot(hist_df["accuracy"], label="treino"); plt.plot(hist_df["val_accuracy"], label="val")
plt.title("Acurácia"); plt.xlabel("Épocas"); plt.ylabel("Acc"); plt.legend(); plt.tight_layout(); plt.savefig("acc_cnn_baseline.png", dpi=150); plt.show()

plt.figure(); plt.plot(hist_df["loss"], label="treino"); plt.plot(hist_df["val_loss"], label="val")
plt.title("Perda (Loss)"); plt.xlabel("Épocas"); plt.ylabel("Loss"); plt.legend(); plt.tight_layout(); plt.savefig("loss_cnn_baseline.png", dpi=150); plt.show()

Opcional) MobileNetV2 TL + FT com 160×160

In [ ]:
base = tf.keras.applications.MobileNetV2(input_shape=(160,160,3), include_top=False, weights="imagenet")
base.trainable = False
tl = tf.keras.Sequential([
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", dtype="float32" if MIXED else None)
], name="mobilenetv2_tl")

tl.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
           loss="sparse_categorical_crossentropy", metrics=["accuracy"])
h1 = tl.fit(train_tf, validation_data=val_tf, epochs=10,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Fine-tuning (últimas ~40 camadas)
base.trainable = True
for layer in base.layers[:-40]:
    layer.trainable = False

tl.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
           loss="sparse_categorical_crossentropy", metrics=["accuracy"])
h2 = tl.fit(train_tf, validation_data=val_tf, epochs=10,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

tl_test_acc = tl.evaluate(test_tf, verbose=0)[1]
print(f"MobileNetV2 (TL+FT) — Acurácia (teste): {tl_test_acc:.4f}")


Limpar arquivo:

In [10]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --to notebook --output limpado.ipynb seu_arquivo.ipynb

[NbConvertApp] WARNING | pattern 'seu_arquivo.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
   